In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-training.csv')
train

In [ ]:
test=pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-test.csv')
test

In [ ]:
train2=train.drop(['Unnamed: 0', 'SeriousDlqin2yrs'], axis=1)
train2

In [ ]:
test2=test.drop(['Unnamed: 0', 'SeriousDlqin2yrs'], axis=1)
test2

In [ ]:
train2=train2.fillna(0)
test2=test2.fillna(0)

In [ ]:
# from sklearn.model_selection import train_test_split
# x_train, x_valid, y_train, y_valid=train_test_split(train2, train['SeriousDlqin2yrs'], random_state=42)
# x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# rf=RandomForestClassifier(n_jobs=4, random_state=42)
# rf.fit(train2, train['SeriousDlqin2yrs'])

In [ ]:
# rf(배깅) -> 데이터 랜덤샘플링, oob / 칼럼마저도 랜덤 _ 어떤 칼럼이 중요한지 파악함 _ 중요한 칼럼만 학습하는..
# 나무를 많이 만드는 혜택이 없음, 다양성을 추구해야 함 // 베이스라인만 해도 커버할 수 있음(이상치~) // 일반화 파워
# 나무를 다양하게 만들어서(칼럼, 데이터 등) -> 앙상블 / 나무들끼리 상관관계가 없음.. 
 
# xgb(부스팅) 비슷하지만 상관관계가 가능 / 전 나무를 적용 / 성능이 좋아진다는 뜻은 train 세세하게 더 복잡하게
# 오버피팅(과대적합) 문제 발생, 따라서 리더보드 점수가 높아지는 것은 아님 / 옵션 설정을 제대로 해야함

# 데이터 분석 순서 / rf(배깅)을 통해 실험, 전처리를 함 -> xgb(부스팅)로 고도화 / 옵션설정만 제대로 하면.. 최상위권  

# tree 모델의 약점_ 앙상블 기법이 대세네

# 러닝레이트 0.3 디폴트 / 그래디언트 y(손실함수, 위험률)=x2 x가 몇일때 가장 최적이냐 -> 기울기가 0일때
# 처음에는 x가 0인 지점을 찾을 수 없음 ri / 데이터에 따라 학습을 함 /  19 <- 20 -> 21 방향에 대한 이해, 19쪽으로 판단하고 줄임
# 얼마나 빠르게 step size-> learing_rate / 0.1 이하로 줄이면 문제가 발생함
# 다차원의 함수 최적의 러닝레이트를 찾는 것이 중요하겠네 / 러닝레이트가 낮아질수록 옵션이 추가

from xgboost import XGBClassifier
xgb=XGBClassifier(learning_rate=0.1)
xgb.fit(train2, train['SeriousDlqin2yrs'])

In [ ]:
# result=rf.predict_proba(test2)
# result

In [ ]:
result=xgb.predict_proba(test2)
result

In [ ]:
len(result)

In [ ]:
result[:,1]

In [ ]:
# test['SeriousDlqin2yrs']=result

In [ ]:
# from sklearn.model_selection import mean_squared_error
# np.sqrt(mean_squared_error(y_valid, result))

In [ ]:
sub=pd.read_csv('/kaggle/input/GiveMeSomeCredit/sampleEntry.csv')
sub

In [ ]:
sub['Probability']=result[:,1]
sub

In [ ]:
sub.to_csv('givemesomecreadit4.csv', index=0)